# Getting started with __htlogicalgates__

Welcome to `htlogicalgates`. This notebook shows you how to create hardware-tailored logical circuits for stabilizer codes using our package.

## Installation

Lorem Ipsum

## How to tailor logical circuits

In [1]:
from htlogicalgates import *